In [1]:
import os, sys
import pyspark
from pyspark.sql import SQLContext

from pyspark.sql import SparkSession # to work with dataframes

sqlContext = SparkSession.builder.appName("test").enableHiveSupport().getOrCreate()

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

import pyspark.sql.functions as F # to work with dataframes siimilar to rdd.map()

import random

from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor

In [2]:
df1 = sqlContext.read.parquet('/home/shashank/Documents/gitWorkspace/SFFD-Spark-Project/Data/FireSmallCleaned.parquet')
df1 = df1.repartition(8)


df1.limit(5).toPandas()

,Unit_ID,Incident_Number,Call_Type,Received_DtTm,On_Scene_DtTm,Available_DtTm,Zipcode_of_Incident,Station_Area,Box,Original_Priority,Call_Type_Group,Number_of_Alarms,Unit_Type,Fire_Prevention_District,Supervisor_District,Location,RowID,Received_month,Received_Hour,response_time,avg_response_history
0,57,18070019,Medical Incident,2018-06-14 06:26:00,2018-06-14 06:39:00,2018-06-14 07:47:00,94122,22,7427,2,Non Life-threatening,1,MEDIC,8,4,"(37.75694617689543, -122.47874036788842)",181653157-57,6,6,13.0,10.571429
1,55,18121635,Medical Incident,2018-10-17 04:34:00,2018-10-17 04:44:00,2018-10-17 05:31:00,94115,10,4154,2,Potentially Life-Threatening,1,MEDIC,5,5,"(37.783357968274046, -122.43860138708125)",182902849-55,10,4,10.0,9.722222
2,61,19026457,Medical Incident,2019-03-03 03:01:00,2019-03-03 03:06:00,2019-03-03 04:10:00,94115,10,4155,3,Potentially Life-Threatening,1,MEDIC,5,5,"(37.78524752592622, -122.43987261265907)",190622092-61,3,3,5.0,7.740741
3,E10,18025399,Medical Incident,2018-03-01 12:21:00,2018-03-01 12:28:00,2018-03-01 12:36:00,94115,10,4235,3,Potentially Life-Threatening,1,ENGINE,5,2,"(37.78386547963962, -122.4413702268871)",180600034-E10,3,12,7.0,10.027027
4,52,18063980,Medical Incident,2018-05-31 03:54:00,2018-05-31 04:02:00,2018-05-31 05:05:00,94107,08,2153,3,Potentially Life-Threatening,1,MEDIC,3,6,"(37.78118776896415, -122.3913514497218)",181512567-52,5,3,8.0,11.684211


# Lets go over each column and decide if we want to include them in the model and how we deal with null values in them.

## Unit_ID

#### This will not be included in training the model since this is not known ahead of time.

## Incident_Number

#### This will not be included in training the model since this is a random number assigned to identify the incident.

## Call_Type

#### This will be included in training the model. Since this is a categorical variable, we will use one hot encoding here.


In [3]:
df1.groupBy('Call_Type').count().show(25), df1.filter(df1['Call_Type'].isNull()).groupBy('Call_Type').count().show(25)

+--------------------+------+
|           Call_Type| count|
+--------------------+------+
|Elevator / Escala...|   976|
|         Marine Fire|    11|
|Confined Space / ...|    44|
|      Administrative|     2|
|              Alarms| 33583|
|Odor (Strange / U...|   255|
|Citizen Assist / ...|  4382|
|              HazMat|   125|
|Watercraft in Dis...|    29|
|           Explosion|    52|
|        Vehicle Fire|   798|
|  Suspicious Package|    24|
|   Train / Rail Fire|    14|
|Extrication / Ent...|    38|
|               Other|  3516|
|        Outside Fire|  3701|
|   Traffic Collision| 11846|
|       Assist Police|    33|
|Gas Leak (Natural...|  1966|
|        Water Rescue|   934|
|   Electrical Hazard|  1344|
|   High Angle Rescue|    24|
|      Structure Fire| 19348|
|Industrial Accidents|    54|
|    Medical Incident|214334|
+--------------------+------+
only showing top 25 rows

+---------+-----+
|Call_Type|count|
+---------+-----+
+---------+-----+



(None, None)

##### Since there are no missing values here, we can just use one hot encoding here.

## Received_DtTm

#### This will not be included in training the model. This is the starting point from which we are predicting the response time.

In [4]:
df1.filter(df1.Received_DtTm.isNull()).count()

0

##### There are no missing values here. Although this won't be used in training, this column has been extracted into other columns which will be used in training.

## On_Scene_DtTm

#### This will not be included in training the model since this is essentially what we are predicting.

In [5]:
df1.filter(df1.On_Scene_DtTm.isNull()).count()

0

##### No missing values here. If there were any missing values here, we will have to drop the row.

## Available_DtTm

#### This will not be included in training the model since this is tied to a unit and there is no way of knowing this ahead of time.

## Zipcode_of_Incident

#### This will be included in training the model.

In [6]:
df1.select('Zipcode_of_Incident').distinct().count()

28

In [7]:
df1.filter(df1.Zipcode_of_Incident.isNull()).count()

118

In [8]:
most_probable_zips = df1.groupBy('Zipcode_of_Incident')\
                        .count().orderBy('count', ascending=False)\
                        .select('Zipcode_of_Incident')\
                        .collect()[:3]

most_probable_zips_list = []
for x in most_probable_zips:
    most_probable_zips_list.append(x[0])
    
df1 = df1.fillna( { 'Zipcode_of_Incident': random.choice(most_probable_zips_list)} )

##### Zipcode is closely related to 'Box' which can be used to fill the missing values. This information will be known at the time of call so zipcode can be used for training the model.

#### Currently filled missing with top 3 zip codes.

## Station_Area

In [9]:
df1.filter(df1.Station_Area.isNull()).count(), df1.select('Station_Area').distinct().count()

(0, 45)

In [10]:
df1.crosstab('Station_Area', 'Zipcode_of_Incident').limit(10).toPandas()

,Station_Area_Zipcode_of_Incident,94102,94103,94104,94105,94107,94108,94109,94110,94111,94112,94114,94115,94116,94117,94118,94121,94122,94123,94124,94127,94129,94130,94131,94132,94133,94134,94158
0,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3072,168,0,0,0,0,0,0,0,0,0,0
1,12,0,0,0,0,0,0,0,0,0,0,71,0,0,2435,17,0,984,0,0,0,0,0,255,0,0,0,0
2,08,0,1099,0,0,5415,0,0,175,78,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,762
3,51,106,0,45,0,0,170,0,0,14,0,0,8,0,0,63,2,2,271,1,0,851,0,0,0,1,0,0
4,19,0,22,0,0,0,0,0,3,0,0,0,0,45,0,0,0,0,0,0,294,0,0,0,4819,0,0,0
5,23,0,0,0,0,0,0,0,0,0,0,0,0,685,0,0,0,2572,0,0,0,0,0,0,38,0,0,0
6,40,0,0,0,0,0,0,0,0,0,0,0,0,2682,0,0,0,603,0,0,11,0,0,0,52,0,0,0
7,09,0,46,0,7,55,0,0,2052,0,0,0,0,0,0,0,0,0,0,1083,0,0,92,0,0,0,4,0
8,15,0,0,0,0,0,0,0,22,0,4581,0,0,0,0,0,0,0,0,0,555,0,0,73,166,0,0,0
9,11,0,0,0,10,0,0,0,4916,0,0,1091,0,0,0,0,0,0,0,0,0,0,0,798,0,0,0,0


In [11]:
x = df1.groupBy('Incident_Number').agg({"Station_Area":"count"}).show(5)

+---------------+-------------------+
|Incident_Number|count(Station_Area)|
+---------------+-------------------+
|       18031325|                  2|
|       18134893|                  2|
|       18052322|                 11|
|       19005278|                  3|
|       18097042|                  2|
+---------------+-------------------+
only showing top 5 rows



#### Although there are no missing values, this will not be included in training the model since the choosen station is based on availability of units and every station responds to multiple zip codes. A single incident can also have multiple stations responding so we just need to predict the response time considering the closest station.

## Box

#### This will not be included in training the model since this is information used only by SFFD for navigation or ease of use.

In [12]:
df1.filter(df1.Box.isNull()).count()

0

## Original_Priority

In [13]:
df1.groupBy('Original_Priority').count().show()

+-----------------+------+
|Original_Priority| count|
+-----------------+------+
|                3|176784|
|             null|     1|
|                E|  7938|
|                B| 14012|
|                C|  8171|
|                A| 22454|
|                1|     2|
|                I|  1269|
|                2| 67931|
+-----------------+------+



#### This will be not included in training the model since this is a value assigned at the time of call and there is no obvious relation with other existing features.

## Call_Type_Group

#### This will be included in training the model. Since this is a categorical variable, we will use one hot encoding here.


In [14]:
df1.groupBy('Call_Type_Group').count().show()

+--------------------+------+
|     Call_Type_Group| count|
+--------------------+------+
|               Alarm| 63327|
|                null|   826|
|Potentially Life-...|146110|
|Non Life-threatening| 79803|
|                Fire|  8496|
+--------------------+------+



In [15]:
df1.crosstab('Call_Type_Group', 'Unit_Type').show()

+-------------------------+-----+------+-----+-------+--------------+------------+-------+-----+
|Call_Type_Group_Unit_Type|CHIEF|ENGINE|MEDIC|PRIVATE|RESCUE CAPTAIN|RESCUE SQUAD|SUPPORT|TRUCK|
+-------------------------+-----+------+-----+-------+--------------+------------+-------+-----+
|                     null|   33|   295|  285|     87|            29|          26|     42|   29|
|     Non Life-threatening|  438| 18202|42915|  15376|           991|         120|   1327|  434|
|                    Alarm|15096| 27213| 1344|    220|           126|         657|     69|18602|
|                     Fire| 1257|  4677|  492|     50|           322|         269|    185| 1244|
|     Potentially Life-...| 1332| 55542|55399|  19344|          5027|         946|   5524| 2996|
+-------------------------+-----+------+-----+-------+--------------+------------+-------+-----+



#### Surprisingly, very few calls to the Fire Department are made because of some fire and more Medic units being requested from Fire Department relative to Fire Engines. 
#### It's safe to assume the missing  Call_Type_Group_Unit_Type as 'Potentially Life-threatening' since that is by far the most occuring 'Call_Type_Group' and the missing Call_Type_Group is less than 1%.

In [16]:
df1 = df1.fillna( { 'Call_Type_Group':'Potentially Life-threatening'} )

## Number_of_Alarms

#### This will be included in training the model.

In [17]:
df1.filter(df1.Number_of_Alarms.isNull()).count()

0

In [18]:
df1.groupBy('Number_of_Alarms').agg({"response_time":"avg"}).show()

+----------------+------------------+
|Number_of_Alarms|avg(response_time)|
+----------------+------------------+
|               1| 9.270422402919786|
|               3|19.896296296296295|
|               4|25.470588235294116|
|               2|16.633217993079583|
+----------------+------------------+



#### There are no missing values and clearly this is a very good indicator of our target.

## Unit_Type

#### This will be included in training the model. Since this is a categorical variable, we will use one hot encoding here.

In [19]:
df1.filter(df1.Unit_Type.isNull()).count()

0

In [20]:
df1.groupBy('Unit_Type').count().show()

+--------------+------+
|     Unit_Type| count|
+--------------+------+
|         MEDIC|100435|
|         CHIEF| 18156|
|  RESCUE SQUAD|  2018|
|RESCUE CAPTAIN|  6495|
|         TRUCK| 23305|
|        ENGINE|105929|
|       SUPPORT|  7147|
|       PRIVATE| 35077|
+--------------+------+



In [21]:
df1.groupBy('Unit_Type').agg({"response_time":"avg"}).show()

+--------------+------------------+
|     Unit_Type|avg(response_time)|
+--------------+------------------+
|         MEDIC|  11.5386468860457|
|         CHIEF| 8.325677461996035|
|  RESCUE SQUAD| 7.797819623389494|
|RESCUE CAPTAIN|11.269745958429562|
|         TRUCK| 7.234842308517486|
|        ENGINE| 6.937165459883507|
|       SUPPORT| 7.779208059325591|
|       PRIVATE|11.799127633492033|
+--------------+------------------+



#### There are no missing values. We can see some association with the target so this could be a good feature. We can also observe the 'MEDIC' and 'ENGINE' are the most common units which can be used in place of any missing values.

## Fire_Prevention_District and Supervisor_District

#### This will be included in training the model. Since this is a categorical variable, we will use one hot encoding here.

In [22]:
df1.filter(df1.Fire_Prevention_District.isNull()).count(), df1.filter(df1.Supervisor_District.isNull()).count()

(0, 0)

In [23]:
df1.groupBy('Fire_Prevention_District').count().show()

+------------------------+-----+
|Fire_Prevention_District|count|
+------------------------+-----+
|                       7|17298|
|                       3|46333|
|                    None| 2074|
|                       8|21489|
|                       5|20351|
|                       6|21699|
|                       9|23086|
|                       1|32744|
|                      10|23356|
|                       4|27538|
|                       2|62594|
+------------------------+-----+



In [24]:
df1.groupBy('Supervisor_District').count().show()

+-------------------+-----+
|Supervisor_District|count|
+-------------------+-----+
|                  7|15515|
|                 11|12894|
|                  3|37872|
|               None|  102|
|                  8|20471|
|                  5|28129|
|                  6|89250|
|                  9|24788|
|                  1|13888|
|                 10|26309|
|                  4|10421|
|                  2|18923|
+-------------------+-----+



In [25]:
df1.groupBy('Supervisor_District').agg({"response_time":"avg"}).show()

+-------------------+------------------+
|Supervisor_District|avg(response_time)|
+-------------------+------------------+
|                  7| 9.988269416693523|
|                 11| 9.632852489530014|
|                  3| 8.979008238276299|
|               None|12.715686274509803|
|                  8|  8.66401250549558|
|                  5|  8.66461658786306|
|                  6| 9.379473389355743|
|                  9|  8.96679845086332|
|                  1| 9.413810483870968|
|                 10| 9.976471929757878|
|                  4|  9.99971211975818|
|                  2| 9.168366538075357|
+-------------------+------------------+



In [26]:
df1.groupBy('Fire_Prevention_District').agg({"response_time":"avg"}).show()

+------------------------+------------------+
|Fire_Prevention_District|avg(response_time)|
+------------------------+------------------+
|                       7| 9.476471268354723|
|                       3| 9.738329052726998|
|                    None|12.217454194792671|
|                       8|10.101121504025315|
|                       5| 8.668271829394133|
|                       6| 8.759482003778976|
|                       9| 9.868794940656676|
|                       1| 9.098369166870267|
|                      10| 9.927812981674945|
|                       4| 8.684835500036314|
|                       2| 8.804549956864875|
+------------------------+------------------+



In [27]:
df1.crosstab('Supervisor_District', 'Fire_Prevention_District').show()

+--------------------------------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+
|Supervisor_District_Fire_Prevention_District|    1|   10|    2|    3|    4|    5|    6|    7|    8|    9|None|
+--------------------------------------------+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+----+
|                                           8|    0|    0| 6490|    0|    0| 4549| 8870|    0|    0|  562|   0|
|                                           4|    0|    0|    0|    0|    0|    0|    0|  175|10246|    0|   0|
|                                          11|    0|    0|    0|    0|    0|    0|    0|    0|    0|12886|   8|
|                                           9|    0| 3262| 7454|    0|    0|    0|12768|    0|    0| 1304|   0|
|                                           5|    0|    0| 8927|    0| 4347|13144|    0|  951|  760|    0|   0|
|                                          10|    0|20094| 1724|  787|    0|    0|   55|    0|    0| 361

#### There is very little differencebetween response times among the different 'Supervisor_District' or 'Fire_Prevention_District' so we could use some educated guess here to fill the nul lvalues. Above cross tabulation shows, most of the missing 'Fire_Prevention_District' falls across Supervisor_District = 6 and when Supervisor_District = 6, probablity of Fire_Prevention_District falls among [2,3,4]. We could replace the nulls in Fire_Prevention_District with on of [2,3,4] and respectively we can in turn use that to replace nulls in Supervisor_District with 6.

In [28]:
df1 = df1.replace('None',None)

missing_Fire_Prevention_District = [2]*7+[3]*8+[4]

df1 = df1.fillna( { 'Fire_Prevention_District':random.choice(missing_Fire_Prevention_District),\
                   'Supervisor_District':6} )

## Location

#### This will not be included in training the model since this accurate location is recorded after the incident has been resolved.

## RowID

#### This will not be included in training the model since this is a random number assigned to identify the row.

## Received_month

#### This will be included in training the model.

## Received_Hour

#### This will be included in training the model.

## response_time

#### This will be included in training the model since this the target we are trying to predict.

In [29]:
df1 = df1.withColumn('label', F.col('response_time'))

## avg_response_history

#### This will be included in training the model. This is a very good indicator of response time and can be easily calculated.

In [32]:
df1 = df1.select(['Call_Type', 'Zipcode_of_Incident', 'Station_Area', 'Received_DtTm',
                  'Box', 'Original_Priority', 'Call_Type_Group', 
                  'Number_of_Alarms', 'Unit_Type', 'Fire_Prevention_District', 
                  'Supervisor_District', 'Received_month', 'Received_Hour', 
                  'response_time', 'avg_response_history', 'label'])

df1.limit(5).toPandas()

,Call_Type,Zipcode_of_Incident,Station_Area,Received_DtTm,Box,Original_Priority,Call_Type_Group,Number_of_Alarms,Unit_Type,Fire_Prevention_District,Supervisor_District,Received_month,Received_Hour,response_time,avg_response_history,label
0,Medical Incident,94112,15,2018-10-15 06:28:00,8466,3,Potentially Life-Threatening,1,ENGINE,9,11,10,6,6.0,7.031250,6.0
1,Medical Incident,94103,08,2018-09-23 08:27:00,0231,A,Non Life-threatening,1,PRIVATE,3,6,9,8,8.0,8.800000,8.0
2,Medical Incident,94109,03,2019-02-27 01:53:00,3223,2,Potentially Life-Threatening,1,MEDIC,4,2,2,1,10.0,9.433333,10.0
3,Medical Incident,94124,25,2019-02-21 01:59:00,6556,2,Potentially Life-Threatening,1,CHIEF,10,10,2,1,20.0,9.173913,20.0
4,Other,94134,42,2018-04-12 05:45:00,6355,B,Potentially Life-Threatening,1,RESCUE CAPTAIN,9,9,4,5,8.0,10.142857,8.0


In [33]:
for i in df1.columns:
    print i," = " , df1.filter(df1[i].isNull()).count()
    
df1.count()

Call_Type  =  0
Zipcode_of_Incident  =  0
Station_Area  =  0
Received_DtTm  =  0
Box  =  0
Original_Priority  =  1
Call_Type_Group  =  0
Number_of_Alarms  =  0
Unit_Type  =  0
Fire_Prevention_District  =  0
Supervisor_District  =  0
Received_month  =  0
Received_Hour  =  0
response_time  =  0
avg_response_history  =  1219
label  =  0


298562

In [35]:
df1 = df1.filter(df1.avg_response_history.isNotNull()).filter(df1.Original_Priority.isNotNull())

In [47]:
train_df = df1.filter(df1['Received_DtTm'] < '2019-01-01 00:00:00' ).drop('Received_DtTm')
print('Train samples = ', train_df.count() )

test_df = df1.filter(df1['Received_DtTm'] >= '2019-01-01 00:00:00' ).drop('Received_DtTm')
print('Test samples = ', test_df.count() )

('Train samples = ', 242879)
('Test samples = ', 54463)


In [48]:
column_vec_in = ['Call_Type','Zipcode_of_Incident','Call_Type_Group',\
                 'Unit_Type','Fire_Prevention_District',\
                 'Supervisor_District','Received_month','Received_Hour']

column_vec_out = ['Call_Type_Vec','Zipcode_of_Incident_Vec','Call_Type_Group_Vec',\
                 'Unit_Type_Vec','Fire_Prevention_District_Vec',\
                 'Supervisor_District_Vec','Received_month_Vec','Received_Hour_Vec']

indexers = [StringIndexer(inputCol=x, outputCol=x+'_indexed') \
           for x in column_vec_in]

encoders = [OneHotEncoder(dropLast=False, inputCol=x+'_indexed', outputCol=y) \
           for x,y in zip(column_vec_in, column_vec_out)]

tmp = [[i,j] for i,j in zip(indexers, encoders)]
tmp = [i for sublist in tmp for i in sublist]

In [49]:
cols_now = ['Call_Type_Vec','Zipcode_of_Incident_Vec','Call_Type_Group_Vec',\
            'Unit_Type_Vec','Fire_Prevention_District_Vec',\
            'Supervisor_District_Vec','Received_month_Vec','Received_Hour_Vec',\
            'avg_response_history']

assembler_features = VectorAssembler(inputCols=cols_now, outputCol='features')

tmp += [assembler_features]
pipeline = Pipeline(stages=tmp)

In [51]:
transformed_train_df = pipeline.fit(train_df).transform(train_df).select(['label','features'])
transformed_train_df.cache()

transformed_test_df = pipeline.fit(test_df).transform(test_df).select(['label','features'])
transformed_test_df.cache()
transformed_test_df.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [52]:
transformed_train_df.count(), transformed_test_df.count()

(242879, 54463)

In [53]:
rf = RandomForestRegressor(numTrees=20, maxDepth=5, seed=2)
model = rf.fit(transformed_train_df)

In [77]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="features",numTrees=20, maxDepth=5, seed=2)

In [78]:
# Train model.  This also runs the indexer.
model = rf.fit(transformed_train_df)

# Make predictions.
predictions = model.transform(transformed_test_df)

In [79]:
print(predictions.columns, predictions.count())
predictions.take(5)

(['label', 'features', 'prediction'], 54463)


Py4JJavaError: An error occurred while calling o2631.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 500.0 failed 1 times, most recent failure: Lost task 0.0 in stage 500.0 (TID 5283, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$11$$anon$1.hasNext(WholeStageCodegenExec.scala:619)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:255)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$2.apply(SparkPlan.scala:247)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitionsInternal$1$$ano

In [76]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)


Py4JJavaError: An error occurred while calling o2455.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 470.0 failed 1 times, most recent failure: Lost task 7.0 in stage 470.0 (TID 5174, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<type:tinyint,size:int,indices:array<int>,values:array<double>>) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$11$$anon$1.hasNext(WholeStageCodegenExec.scala:619)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11

In [ ]:
rfModel = model.stages[1]
print(rfModel)  # summary only